In [1]:
import torch
import math
import numpy as np
import scipy.io as sci
import torch.nn as nn
import os
import h5py
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as Data
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
import torch
from torch import nn
from torch import optim
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score
from nltk.classify.megam import numpy
from scipy import interp
from itertools import cycle
import time

In [3]:
import numpy as np
import os
import math
import random
import torch
# from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA
from scipy import stats
import plotly.graph_objects as go

from sklearn.utils import shuffle
import torch.nn as nn
!pip install torchsummary
import matplotlib.pyplot as plt
from torchsummary import summary
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

You should consider upgrading via the '/Users/pro/Documents/Sklearn/venv/bin/python -m pip install --upgrade pip' command.


In [4]:
import numpy as np
import seaborn
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam
from torchvision import datasets, transforms

USE_CUDA = True

In [5]:
train_set = pd.read_csv('../capsule_code/data/nc_train_0.csv', sep=' ',header=None,names=['0','1','2','3','4','5','6','label'])
train_set

,0,1,2,3,4,5,6,label
0,1.168561,-0.907673,1.168743,0.732893,-0.307202,0.578619,-0.399994,0
1,-0.064005,-1.017375,-0.965043,0.446558,-0.307202,0.437597,-0.399994,0
2,1.168561,-1.675586,-1.365127,0.160224,-0.307202,0.617291,-0.399994,7
3,1.168561,0.079644,1.168743,1.591895,-0.265008,0.862663,-0.399994,10
4,-1.296572,0.189345,-1.098404,-0.126110,-0.349396,-2.416853,-0.477236,3
...,...,...,...,...,...,...,...,...
11195,1.168561,1.615469,1.435466,0.446558,-0.349396,-0.063482,-0.399994,14
11196,1.168561,1.615469,1.435466,0.732893,-0.349396,-0.062816,-0.399994,14
11197,1.168561,1.615469,1.435466,1.019227,-0.349396,-0.062816,-0.399994,14
11198,1.168561,1.615469,1.435466,1.305561,-0.349396,-0.062482,-0.399994,14


In [6]:
test_set = pd.read_csv('../capsule_code/data/nc_test_0.csv', sep=' ',header=None,names=['0','1','2','3','4','5','6','label'])
test_set

,0,1,2,3,4,5,6,label
0,-1.296572,-0.688269,-0.831681,-0.412444,-0.307202,0.306243,-0.361374,0
1,-0.064005,-1.456182,-1.231766,-0.985112,-0.349396,-2.416853,-0.477236,3
2,1.168561,-0.578567,0.235212,-1.557780,-0.307202,0.491938,-0.361374,0
3,1.168561,-1.675586,-0.431596,-1.271446,-0.307202,0.627293,-0.399994,7
4,1.168561,-1.675586,-1.098404,1.305561,-0.307202,0.618958,-0.399994,7
...,...,...,...,...,...,...,...,...
2795,-1.296572,-0.030058,0.635296,1.019227,-0.138425,0.339581,-0.245511,12
2796,1.168561,0.847556,-1.231766,1.305561,-0.349396,-0.057481,-0.399994,11
2797,-1.296572,0.408749,0.635296,-0.412444,-0.265008,0.355584,-0.322753,0
2798,-0.064005,1.286364,-0.698319,0.732893,-0.307202,0.539946,-0.399994,0


In [7]:
# 划分训练集和验证集
train_set = shuffle(train_set)
all_train_data = shuffle(train_set)
train_data = all_train_data[['0','1','2','3','4','5','6']]
train_label = all_train_data['label']

X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_label, test_size = 0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [8]:
# 测试集
test_set = shuffle(test_set)
all_test_data = shuffle(test_set)
X_test = all_test_data[['0','1','2','3','4','5','6']]
y_test = all_test_data['label']

In [9]:
X_train, X_test, X_valid = np.array(X_train).reshape(-1, 7), np.array(X_test).reshape(-1, 7), np.array(X_valid).reshape(-1, 7)
y_train, y_test, y_valid = np.array(y_train).reshape(-1, 1), np.array(y_test).reshape(-1, 1), np.array(y_valid).reshape(-1, 1)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train, X_valid, X_test = torch.FloatTensor(X_train), torch.FloatTensor(X_valid), torch.FloatTensor(X_test)
y_train, y_valid, y_test = torch.FloatTensor(y_train), torch.FloatTensor(y_valid), torch.FloatTensor(y_test)

In [11]:
from torch.utils.data import Dataset, DataLoader

class Dataset(Dataset):
  # 'Characterizes a dataset for PyTorch'
    def __init__(self, data, labels):
        # 'Initialization'
        self.labels = labels
        self.data = data

    def __len__(self):
        # 'Denotes the total number of samples'
        return len(self.data)

    def __getitem__(self, index):
        # 'Generates one sample of data'
        # Select sample
        return self.data[index], self.labels[index]

In [12]:
config = {
    'batch_size': 100,
    'shuffle': True,
    'drop_last': False
}

In [13]:
train_set = Dataset(X_train, y_train)
train_loader = DataLoader(
    train_set,
    batch_size=config['batch_size'],
    shuffle=config['shuffle'],
#     num_workers=config['num_workers'],
    drop_last=config['drop_last']
)

validation_set = Dataset(X_valid, y_valid)
validation_loader = DataLoader(
    validation_set,
    batch_size=config['batch_size'],
    shuffle=config['shuffle'],
#     num_workers=config['num_workers'],
    drop_last=config['drop_last']
)

test_set = Dataset(X_test, y_test)
test_loader = DataLoader(
    test_set,
    batch_size=config['batch_size'],
    shuffle=config['shuffle'],
#     num_workers=config['num_workers'],
    drop_last=config['drop_last']
)

### CNN 

In [84]:
class CNN(nn.Module):
    
    def __init__(self):
        '''Constructs the ConvLayer with a specified input and output size.
           param in_channels: input depth of an image, default value = 1
           param out_channels: output depth of the convolutional layer, default value = 256
           '''
        super(CNN, self).__init__()

        # defining a convolutional layer of the specified size
        # torch.Size([100, 64, 3])
        self.conv1 = nn.Conv1d(in_channels=7, out_channels=64, 
                              kernel_size=3, stride=1, padding=2)
        # torch.Size([100, 128, 5])
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, 
                              kernel_size=3, stride=1, padding=2)
        # torch.Size([100, 256, 7])
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, 
                              kernel_size=3, stride=1, padding=2)
        # torch.Size([100, 512, 9])
        self.conv4 = nn.Conv1d(in_channels=256, out_channels=512, 
                              kernel_size=3, stride=1, padding=2)
        
        self.fc1 = nn.Linear(512*9, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 16)
#         self.conv5 = nn.Conv1d(in_channels=512, out_channels=1024, 
#                               kernel_size=3, stride=1, padding=2)
#         self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        '''Defines the feedforward behavior.
           param x: the input to the layer; an input image
           return: a relu-activated, convolutional layer
           '''
        # 将 ReLu 激活应用于 conv 层的输出
        features = F.relu(self.conv1(x))
#         print(features.shape)
        features = F.relu(self.conv2(features))
        features = F.relu(self.conv3(features))
        features = F.relu(self.conv4(features))
        features = features.view(-1, 512*9)
        features = F.relu(self.fc1(features))
        features = F.relu(self.fc2(features))
        features = F.relu(self.fc3(features))
        features = self.fc4(features)
#         features = self.conv5(features)
#         features = F.relu(features) # will have dimensions (batch_size, 20, 20, 256)
        return features

### Attention 

In [15]:
class Attention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1, dropout = 0.3, max_length = T):
        super(Attention, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.max_length = max_length
        self.output_size = output_size
        self.dropout = dropout

        self.gru = nn.GRU(input_size, hidden_size, self.n_layers, bidirectional = True)
        self.attn = nn.Linear(self.hidden_size*2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size*2, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input):
        
        output, hidden = self.gru(input)
        output = output.transpose(0,1)
        hidden = torch.cat((hidden[0],hidden[1]),1)

        attn_weights = self.attn(hidden)
        attn_weights = torch.bmm(attn_weights.unsqueeze(1), output)
        attn_weights = F.softmax(self.attn(attn_weights.squeeze(1)))

        attn_applied = torch.bmm(attn_weights.unsqueeze(1), output)
        attn_applied = attn_applied.squeeze(1)
        output = self.attn_combine(attn_applied)
        output = F.relu(output)
        output = self.out(output)
        
        return output

NameError: name 'T' is not defined

In [14]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_szie1, hidden_szie2, output_size):
        super(RNN, self).__init__()
        
        self.rnn = nn.LSTM(input_size, hidden_szie1, 1, dropout = 0.3)
        self.r2h = nn.Linear(hidden_szie1, hidden_szie2)
        self.h2o = nn.Linear(hidden_szie2, output_size)

    def forward(self, input):
        hidden, _ = self.rnn(input)
        fc1 = F.relu(self.r2h(hidden[T-1]))
        output = self.h2o(fc1)
        #output = F.softmax(output)

        return output 

In [ ]:
rnn = RNN(input_size=7, hidden_szie1=100, hidden_szie2=100, output_size=16)
